In [1]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(".."))
from core.viz import plot_bar, plot_dynamic_trends, plot_line, plot_corr_triangle, plot_scatter
from core.s3 import S3AssetManager

In [2]:
notebook_name = "fortidex_harina_camaron"
s3 = S3AssetManager(notebook_name=notebook_name)

In [3]:
control = pd.read_excel(
    "../raw/fortidex/HUMEDADES EN LINEA PRO CAMARÓN 2025.xlsb",
     sheet_name="CLEAR",
     #skiprows=8,
     engine="pyxlsb")

In [6]:
control['FECHA'] = pd.to_numeric(control['FECHA'], errors='coerce')
#control['FECHA'] = pd.to_datetime(control['FECHA'], unit='D')
control['HORA'] = pd.to_timedelta(control['HORA'], unit='D', errors='coerce')
control = control[control["FECHA"].notna()]

In [7]:
control

,FECHA,HORA,TURNO,LOTE,PALLET,PT_25,PT_50,REPROCESO_25,REPROCESO_50,OBSERVACIÓN,...,PPM,FLUJO,VARIADOR,NOMBRE,SIPERNAT_SI,SIPERNAT_NO,INIMHOL_INICIO_KG,INIMHOL_FIN_KG,INIMHOL_TOTAL_KG,INIMHOL_DOSIS_KG/TON
0,45674.0,0 days 02:55:00.000000001,1.0,4135-01-2025,NaN,NaN,NaN,46.0,NaN,LIMPIEZA DEL SISTEMA,...,530.434783,22.0,60.0,GREENOX,NaN,ü,118.00,116.15,1.85,1.608696
1,45674.0,0 days 04:00:00.000000002,1.0,4135-01-2025,1--2,140.0,NaN,NaN,NaN,NaN,...,1811.428571,19.0,40.0,GREENOX,NaN,ü,116.15,111.60,4.55,1.3
2,45674.0,0 days 06:20:00,1.0,4135-01-2025,3--4 + 50 SACOS,190.0,NaN,NaN,NaN,NaN,...,1705.263158,19.0,40.0,GREENOX,NaN,ü,205.85,203.75,2.10,2.0
3,45675.0,0 days 01:20:00.000000003,1.0,4135-01-2025,NaN,NaN,NaN,42.0,NaN,LIMPIEZA DEL SISTEMA,...,390.47619,22.0,60.0,GREENOX,NaN,ü,203.75,201.80,1.95,1.695652
4,45675.0,0 days 01:45:00.000000002,1.0,4135-01-2025,NaN,NaN,NaN,46.0,NaN,HARINA SECA,...,1895.652174,25.0,45.0,GREENOX,NaN,ü,201.80,195.35,6.45,1.228571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,45927.0,0 days 20:10:00.000000001,2.0,07-270-2025,6--7--8 + 44 SACOS,NaN,NaN,254.0,NaN,NaN,...,840.944882,21.0,45.0,GREENOX,NaN,ü,NaN,NaN,0.00,0x7
866,45928.0,0 days 20:50:00.000000003,1.0,71-271-2025,NaN,NaN,NaN,NaN,19.0,LIMPIEZA SISTEMA (PESCADO),...,357.894737,20.0,60.0,GREENOX,NaN,ü,NaN,NaN,0.00,0x7
867,45928.0,0 days 21:00:00,1.0,72-271-2025,NaN,NaN,NaN,NaN,17.0,LIMPIEZA SISTEMA (CAMARÓN),...,282.352941,20.0,60.0,GREENOX,NaN,ü,NaN,NaN,0.00,0x7
868,45928.0,0 days 22:30:00,1.0,07-271-2025,1--2,NaN,NaN,140.0,NaN,NaN,...,831.428571,22.0,45.0,GREENOX,NaN,ü,NaN,NaN,0.00,0x7


In [8]:
control.columns

Index(['FECHA', 'HORA', 'TURNO', 'LOTE', 'PALLET', 'PT_25', 'PT_50',
       'REPROCESO_25', 'REPROCESO_50', 'OBSERVACIÓN', 'HUM', 'PROTEINA',
       'GRASA', 'CENIZA', 'GRANULOMETRIA', 'TEMP', 'TEMP_POND', 'TVN',
       'CONCENTRADO', 'COLOR', 'DOSIS', 'PT_SACOS', 'PT_TON',
       'ANTIOXIDANTE_INICIO_KG', 'ANTIOXIDANTE_FIN_KG',
       'ANTIOXIDANTE_TOTAL_KG', 'DOSIS_KG_TON', 'PPM', 'FLUJO', 'VARIADOR',
       'NOMBRE', 'SIPERNAT_SI', 'SIPERNAT_NO', 'INIMHOL_INICIO_KG',
       'INIMHOL_FIN_KG', 'INIMHOL_TOTAL_KG', 'INIMHOL_DOSIS_KG/TON'],
      dtype='object')

In [9]:
cols_num = [
'PT_25', 'PT_50',
'REPROCESO_25', 'REPROCESO_50',
'HUM', 'PROTEINA',
'GRASA', 'CENIZA', 'GRANULOMETRIA',
'TEMP', 'TEMP_POND', 
'COLOR', 'DOSIS', 'PT_SACOS', 'PT_TON',
'ANTIOXIDANTE_INICIO_KG', 
'ANTIOXIDANTE_FIN_KG',
'ANTIOXIDANTE_TOTAL_KG',
'DOSIS_KG_TON', 'PPM', 'FLUJO', 'VARIADOR',
'INIMHOL_INICIO_KG',
'INIMHOL_FIN_KG',
'INIMHOL_TOTAL_KG',
  'INIMHOL_DOSIS_KG/TON'
]

In [10]:
for cl_num in cols_num:
    control[cl_num] = pd.to_numeric(control[cl_num], errors="coerce")

In [11]:
control

,FECHA,HORA,TURNO,LOTE,PALLET,PT_25,PT_50,REPROCESO_25,REPROCESO_50,OBSERVACIÓN,...,PPM,FLUJO,VARIADOR,NOMBRE,SIPERNAT_SI,SIPERNAT_NO,INIMHOL_INICIO_KG,INIMHOL_FIN_KG,INIMHOL_TOTAL_KG,INIMHOL_DOSIS_KG/TON
0,45674.0,0 days 02:55:00.000000001,1.0,4135-01-2025,NaN,NaN,NaN,46.0,NaN,LIMPIEZA DEL SISTEMA,...,530.434783,22.0,60.0,GREENOX,NaN,ü,118.00,116.15,1.85,1.608696
1,45674.0,0 days 04:00:00.000000002,1.0,4135-01-2025,1--2,140.0,NaN,NaN,NaN,NaN,...,1811.428571,19.0,40.0,GREENOX,NaN,ü,116.15,111.60,4.55,1.300000
2,45674.0,0 days 06:20:00,1.0,4135-01-2025,3--4 + 50 SACOS,190.0,NaN,NaN,NaN,NaN,...,1705.263158,19.0,40.0,GREENOX,NaN,ü,205.85,203.75,2.10,2.000000
3,45675.0,0 days 01:20:00.000000003,1.0,4135-01-2025,NaN,NaN,NaN,42.0,NaN,LIMPIEZA DEL SISTEMA,...,390.476190,22.0,60.0,GREENOX,NaN,ü,203.75,201.80,1.95,1.695652
4,45675.0,0 days 01:45:00.000000002,1.0,4135-01-2025,NaN,NaN,NaN,46.0,NaN,HARINA SECA,...,1895.652174,25.0,45.0,GREENOX,NaN,ü,201.80,195.35,6.45,1.228571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,45927.0,0 days 20:10:00.000000001,2.0,07-270-2025,6--7--8 + 44 SACOS,NaN,NaN,254.0,NaN,NaN,...,840.944882,21.0,45.0,GREENOX,NaN,ü,NaN,NaN,0.00,NaN
866,45928.0,0 days 20:50:00.000000003,1.0,71-271-2025,NaN,NaN,NaN,NaN,19.0,LIMPIEZA SISTEMA (PESCADO),...,357.894737,20.0,60.0,GREENOX,NaN,ü,NaN,NaN,0.00,NaN
867,45928.0,0 days 21:00:00,1.0,72-271-2025,NaN,NaN,NaN,NaN,17.0,LIMPIEZA SISTEMA (CAMARÓN),...,282.352941,20.0,60.0,GREENOX,NaN,ü,NaN,NaN,0.00,NaN
868,45928.0,0 days 22:30:00,1.0,07-271-2025,1--2,NaN,NaN,140.0,NaN,NaN,...,831.428571,22.0,45.0,GREENOX,NaN,ü,NaN,NaN,0.00,NaN


In [12]:
control[['OBSERVACIÓN', 'HUM', 'PROTEINA', 'COLOR', "TEMP"]]


,OBSERVACIÓN,HUM,PROTEINA,COLOR,TEMP
0,LIMPIEZA DEL SISTEMA,4.90,NaN,4.0,39.0
1,NaN,6.79,51.35,3.5,41.2
2,NaN,8.22,50.25,3.0,41.0
3,LIMPIEZA DEL SISTEMA,4.30,NaN,4.0,40.0
4,HARINA SECA,4.62,NaN,4.0,40.2
...,...,...,...,...,...
865,NaN,8.08,NaN,4.0,40.9
866,LIMPIEZA SISTEMA (PESCADO),6.45,NaN,5.0,39.9
867,LIMPIEZA SISTEMA (CAMARÓN),7.98,NaN,4.5,39.7
868,NaN,9.30,NaN,4.0,40.2


In [13]:
control['OBSERVACIÓN'].value_counts(dropna=False)

OBSERVACIÓN
NaN                                                 616
LIMPIEZA DEL SISTEMA                                 84
LIMPIEZA SISTEMA (CAMARÓN)                           72
LIMPIEZA SISTEMA (PESCADO)                           64
PROTEINA BAJA                                         6
TONO 5                                                5
TONO 4                                                4
SECADO MANUAL                                         4
HUMEDAD ALTA                                          2
PROTEÍNA BAJA                                         2
SECADO AUTOMATICO                                     2
TONO 6                                                1
GRANULOMETRIA ALTA                                    1
HUMEDOS                                               1
TONO 3,5 - PROTEÍNA BAJA                              1
TONO 4 - PROTEÍNA BAJA                                1
HARINA SECA                                           1
GRANOLUMETRIA ALTA SE PASO  POR EL M

In [14]:
control[control["COLOR"]<=3]


,FECHA,HORA,TURNO,LOTE,PALLET,PT_25,PT_50,REPROCESO_25,REPROCESO_50,OBSERVACIÓN,...,PPM,FLUJO,VARIADOR,NOMBRE,SIPERNAT_SI,SIPERNAT_NO,INIMHOL_INICIO_KG,INIMHOL_FIN_KG,INIMHOL_TOTAL_KG,INIMHOL_DOSIS_KG/TON
2,45674.0,0 days 06:20:00,1.0,4135-01-2025,3--4 + 50 SACOS,190.0,NaN,NaN,NaN,NaN,...,1705.263158,19.0,40.0,GREENOX,NaN,ü,205.85,203.75,2.10,2.000000
25,45695.0,0 days 14:44:59.999999997,2.0,07-038-2025,5--6,NaN,NaN,140.0,NaN,PROTEÍNA BAJA,...,1268.571429,19.0,40.0,GREENOX,NaN,ü,135.55,133.50,2.05,1.518519
26,45695.0,0 days 17:30:00.000000002,2.0,07-038-2025,7--8--9 + 44 SACOS,NaN,NaN,254.0,NaN,PROTEÍNA BAJA,...,1250.393701,19.0,40.0,GREENOX,NaN,ü,133.40,131.20,2.20,1.257143
28,45696.0,0 days 07:00:00.000000002,2.0,07-039-2025,1--2,NaN,NaN,140.0,NaN,PROTEINA BAJA,...,1668.571429,19.0,40.0,GREENOX,NaN,ü,129.05,124.95,4.10,1.171429
29,45696.0,0 days 07:59:59.999999997,2.0,07-039-2025,3--4,NaN,NaN,140.0,NaN,PROTEINA BAJA,...,1680.000000,19.0,40.0,GREENOX,NaN,ü,124.95,119.10,5.85,1.147059
30,45696.0,0 days 09:00:00,1.0,07-039-2025,5--6,NaN,NaN,140.0,NaN,PROTEINA BAJA,...,1708.571429,19.0,40.0,GREENOX,NaN,ü,119.60,116.25,3.35,2.791667
31,45696.0,0 days 09:45:00,1.0,07-039-2025,7--8,NaN,NaN,140.0,NaN,PROTEINA BAJA,...,1720.000000,19.0,40.0,GREENOX,NaN,ü,116.25,112.20,4.05,1.157143
32,45696.0,0 days 10:30:00,1.0,07-039-2025,9--10,NaN,NaN,140.0,NaN,PROTEINA BAJA,...,1714.285714,19.0,40.0,GREENOX,NaN,ü,112.20,105.90,6.30,1.285714
33,45696.0,0 days 11:50:00.000000003,1.0,07-039-2025,11--12 + 4 SACOS,NaN,NaN,144.0,NaN,PROTEINA BAJA,...,1694.444444,19.0,40.0,GREENOX,NaN,ü,162.95,161.40,1.55,1.550000
124,45736.0,0 days 10:45:00.000000002,1.0,07-079-2025,9---10++2 SACOS,NaN,NaN,142.0,NaN,NaN,...,1921.126761,22.0,42.0,GREENOX,NaN,ü,57.20,53.05,4.15,1.317460


In [17]:
control.groupby('OBSERVACIÓN', dropna=False).agg(
    protein_avg=('PROTEINA', 'mean'),
    protein_median=('PROTEINA', 'median'),
    moisture_avg=('HUM', 'mean'),
    moisture_median=('HUM', 'median'),
    temp_avg=('TEMP', 'mean'),
    temp_median=('TEMP', 'median'),
    color_avg=('COLOR', 'mean'),
    color_median=('COLOR', 'median'),
    grasa_avg=( 'GRASA', 'mean'),
    grasa_median=('GRASA', 'median'),
    cenizas_avg=('CENIZA', 'mean'),
    cenizas_median=('CENIZA', 'median'),
    granulometria_avg=('GRANULOMETRIA', 'mean'),
    granulometria_median=('GRANULOMETRIA', 'median'),
    antioxidante_avg=('ANTIOXIDANTE_TOTAL_KG', 'mean'),
    antioxidante_median=('ANTIOXIDANTE_TOTAL_KG', 'median'),
)

,protein_avg,protein_median,moisture_avg,moisture_median,temp_avg,temp_median,color_avg,color_median,grasa_avg,grasa_median,cenizas_avg,cenizas_median,granulometria_avg,granulometria_median,antioxidante_avg,antioxidante_median
OBSERVACIÓN,,,,,,,,,,,,,,,,
GRANOLUMETRIA ALTA SE PASO POR EL MOLINO GRANDE,NaN,NaN,9.060000,9.060,41.900000,41.90,4.000000,4.0,NaN,NaN,NaN,NaN,45.450000,45.450,20.850000,20.850
GRANULOMETRIA ALTA,NaN,NaN,6.050000,6.050,39.900000,39.90,4.000000,4.0,NaN,NaN,NaN,NaN,53.310000,53.310,9.350000,9.350
HARINA SECA,NaN,NaN,4.620000,4.620,40.200000,40.20,4.000000,4.0,NaN,NaN,NaN,NaN,27.120000,27.120,5.450000,5.450
HUMEDAD ALTA,NaN,NaN,10.170000,10.170,40.650000,40.65,4.000000,4.0,NaN,NaN,NaN,NaN,44.355000,44.355,9.100000,9.100
HUMEDOS,NaN,NaN,10.350000,10.350,39.900000,39.90,4.000000,4.0,NaN,NaN,NaN,NaN,32.450000,32.450,11.950000,11.950
LIMPIEZA DEL SISTEMA,54.450000,54.450,7.621829,7.720,39.324390,39.40,4.541667,4.5,NaN,NaN,NaN,NaN,34.519878,34.085,4.602976,4.700
LIMPIEZA SISTEMA (CAMARÓN),49.370000,49.370,7.772361,7.700,39.501389,39.60,4.840278,4.5,NaN,NaN,NaN,NaN,33.188333,31.775,2.580556,2.250
LIMPIEZA SISTEMA (PESCADO),54.305000,54.305,7.225781,7.355,39.303906,39.40,4.859375,5.0,NaN,NaN,NaN,NaN,30.975781,30.480,2.424531,2.075
PROTEINA BAJA,49.103333,49.245,8.025000,7.975,40.500000,40.60,3.000000,3.0,NaN,NaN,NaN,NaN,37.611667,37.150,14.925000,14.975


In [50]:
plot_scatter(df=control,x_col='COLOR',y_col='TEMP')

In [45]:
plot_corr_triangle(
df=control[['HUM', 'COLOR']],
value_cols=['HUM', 'COLOR'],
 method='pearson', 
    title='<b>Correlación (Pearson) — triángulo superior</b>',
     decimals=2, width=1000, height=400, )

ValueError: Invalid property specified for object of type plotly.graph_objs.heatmap.ColorBar: 'titleside'

Did you mean "title"?

    Valid properties:
        bgcolor
            Sets the color of padded area.
        bordercolor
            Sets the axis line color.
        borderwidth
            Sets the width (in px) or the border enclosing this
            color bar.
        dtick
            Sets the step in-between ticks on this axis. Use with
            `tick0`. Must be a positive number, or special strings
            available to "log" and "date" axes. If the axis `type`
            is "log", then ticks are set every 10^(n*dtick) where n
            is the tick number. For example, to set a tick mark at
            1, 10, 100, 1000, ... set dtick to 1. To set tick marks
            at 1, 100, 10000, ... set dtick to 2. To set tick marks
            at 1, 5, 25, 125, 625, 3125, ... set dtick to
            log_10(5), or 0.69897000433. "log" has several special
            va

In [35]:
plot_corr_triangle

<function core.viz.plot_corr_triangle(df: pandas.core.frame.DataFrame, value_cols: Sequence[str], *, method: str = 'pearson', title: str = 'Correlación (Pearson) — triángulo superior', decimals: int = 2, width: int = 750, height: int = 650, font_family: str = 'Inter, Arial, sans-serif', salida_html: Optional[str] = None) -> plotly.graph_objs._figure.Figure>